TODO: Until we get real data, make synthetic data

#### Temporary: Synthetic Data

In [1]:
# Synthetic Data
import numpy as np
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=200, n_features=200, random_state=42)
data = np.concatenate((X, y.reshape(-1, 1)), axis = 1)

### Preprocessing

In [2]:
import preprocess
np.random.seed(42)
states = np.random.randint(low = 0, high = 1000000, size=(100,)) # numpy array with our 100 random states
data_splits = {}
for rst in states:
    data_splits[rst] = preprocess.preprocess(data, rst, selection=None, extraction='umap', k=5)
    # k: [1, 10, 20, ..., 120, 130, 140, 150, 160]

c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\User

### Modeling

In [3]:
import modeling as models
model_names = ["dt", "rf", "gb", "xgb", "lgb", "et", "ab", "lr", "lr1", "lr2", "lre", "lsv", "nlsv", "knn", "lda", "gnb", "mlp"]


model_res = {} # key as model type, then sub dictionary with rst as key and predicted classes & model itself ex: model_res["dt"][rst] --> (y_pred_dt, dt) 
for model in model_names:
    model_res[model] = {}

for state in data_splits.keys():    
    model_res["dt"][state] = models.model_decisiontree(*data_splits[state]) # decision tree
    model_res["rf"][state] = models.model_randomforest(*data_splits[state]) # random forest
    model_res["gb"][state] = models.model_gradientboosting(*data_splits[state]) # grad boosting
    model_res['xgb'][state] = models.model_extremegb(*data_splits[state])
    model_res["lgb"][state] = models.model_lightgb(*data_splits[state])
    model_res["et"][state] = models.model_extratrees(*data_splits[state])
    model_res["ab"][state] = models.model_adaboost(*data_splits[state])
    model_res["lr"][state] = models.model_logisticregression(*data_splits[state])
    model_res["lr1"][state] = models.model_lassoregularization(*data_splits[state])
    model_res["lr2"][state] = models.model_ridgeRegularization(*data_splits[state])
    model_res["lre"][state] = models.model_elasticNetRegularization(*data_splits[state])
    model_res["lsv"][state] = models.model_linearSupportVector(*data_splits[state])
    model_res["nlsv"][state] = models.model_nonLinearSupportVector(*data_splits[state])
    model_res["knn"][state] = models.model_kNearestNeighbor(*data_splits[state])
    model_res["lda"][state] = models.model_linearDiscriminantAnalysis(*data_splits[state])
    model_res["gnb"][state] = models.model_gaussianNaiveBayes(*data_splits[state])
    model_res["mlp"][state] = models.model_multiLayerPerceptron(*data_splits[state])

c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklea

### Evaluation

In [4]:
# basic view for now to see some metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from metrics import getMetrics
model_metrics = {}
for modelName in model_names:
    model_metrics[modelName] = {}
    model_metrics[modelName]['auc'] = 0.0
    model_metrics[modelName]['acc'] = 0.0
    model_metrics[modelName]['precision'] = 0.0
    model_metrics[modelName]['recall'] = 0.0
    model_metrics[modelName]['f1'] = 0.0

# calculate the metrics for all models
for rst in data_splits.keys():
    for model in model_names:
        metric = getMetrics(data_splits[rst][3], data_splits[rst][1], model_res[model][rst][0], model_res[model][rst][1])
        model_metrics[model]['auc'] += metric[0]
        model_metrics[model]['acc'] += metric[1]
        model_metrics[model]['precision'] += metric[2]
        model_metrics[model]['recall'] += metric[3]
        model_metrics[model]['f1'] += metric[4]

# calculate the averages for each model
model_averages = {}
for model in model_names:
    model_averages[model] = {}
    model_averages[model]['auc'] = model_metrics[model]['auc'] / len(data_splits)
    model_averages[model]['acc'] = model_metrics[model]['acc'] / len(data_splits)
    model_averages[model]['precision'] = model_metrics[model]['precision'] / len(data_splits)
    model_averages[model]['recall'] = model_metrics[model]['recall'] / len(data_splits)
    model_averages[model]['f1'] = model_metrics[model]['f1'] / len(data_splits)
print(model_averages)


c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\maxep\Documents\Github Repos\SeniorProj

{'dt': {'auc': np.float64(0.5369444444444447), 'acc': 0.5327500000000001, 'precision': 0.4875349078863339, 'recall': 0.5788888888888891, 'f1': 0.5238123187701658}, 'rf': {'auc': np.float64(0.5722474747474751), 'acc': 0.564, 'precision': 0.5169558847600706, 'recall': 0.6011111111111113, 'f1': 0.5527461162150706}, 'gb': {'auc': np.float64(0.5555808080808083), 'acc': 0.5479999999999999, 'precision': 0.5008302810869005, 'recall': 0.5833333333333333, 'f1': 0.5349907266597529}, 'xgb': {'auc': np.float64(0.5583838383838388), 'acc': 0.5372500000000001, 'precision': 0.4910855069154232, 'recall': 0.5833333333333335, 'f1': 0.529818594411748}, 'lgb': {'auc': np.float64(0.5647348484848487), 'acc': 0.5417499999999998, 'precision': 0.4954624349465572, 'recall': 0.6088888888888888, 'f1': 0.5430585084625342}, 'et': {'auc': np.float64(0.5769444444444444), 'acc': 0.5597499999999997, 'precision': 0.5100642059649867, 'recall': 0.6233333333333337, 'f1': 0.5591207674597372}, 'ab': {'auc': np.float64(0.571755

c:\Users\maxep\Documents\Github Repos\SeniorProject\TheBioBros\.conda\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [5]:
# Barchart (models)